# Dataset Maestro por CCAA (salario mediano Decil 5)

In [52]:
import pandas as pd

# Lectura de archivos CSV
precios_df = pd.read_csv("../data/clean/precios_sql.csv")
salario_ccaa_df = pd.read_csv("../data/clean/salario_ccaa_sql.csv")
superficie_df = pd.read_csv("../data/clean/superficie_extendida.csv")
interes_df = pd.read_csv("../data/clean/interes_fijo_sql.csv")

In [24]:
# Filtrar salario por comunidad autónoma: decil 5 y jornada Total (excluyendo Total Nacional)
salario_ccaa_filtered = salario_ccaa_df[
    (salario_ccaa_df['tipo_de_jornada'] == 'Total') &
    (salario_ccaa_df['decil'] == '5') &
    (salario_ccaa_df['comunidades_y_ciudades_autónomas'] != 'Total Nacional')
].copy()
salario_ccaa_filtered.rename(columns={
    'comunidades_y_ciudades_autónomas': 'region',
    'año': 'year',
    'total': 'salario_mensual'
}, inplace=True)
salario_ccaa_filtered = salario_ccaa_filtered[['region', 'year', 'salario_mensual']]




In [25]:
# Calcular interés medio anual (tipo fijo, naturaleza Vivienda)
interes_filtered = interes_df[
    (interes_df['tipo_de_interés'] == 'Fijo') &
    (interes_df['naturaleza_de_la_finca'] == 'Viviendas')
]
interest_by_year = interes_filtered.groupby('año', as_index=False)['total'].mean()
interest_by_year.rename(columns={'año': 'year', 'total': 'interes_medio'}, inplace=True)

# Calcular superficie útil promedio por comunidad (m²) a partir de la distribución de tamaños de vivienda
# Mapeo de categorías de superficie a valores representativos en m²
categoria_a_m2 = {
    'Menos de 46 m2': 40,
    'Entre 46 y 75 m2': 60,
    'Entre 76 y 105 m2': 90,
    'Entre 106 y 150 m2': 128,
    'Más de 150 m2': 180
}


In [26]:
# Filtrar datos de superficie para tamaño_hogar Total, excluyendo la fila 'Total' consolidada
sup_filt = superficie_df[
    (superficie_df['tamaño_hogar'] == 'Total') & 
    (superficie_df['superficie_util'] != 'Total')
].copy()
sup_filt['rep_area'] = sup_filt['superficie_util'].map(categoria_a_m2)
# Calcular promedio ponderado de superficie por comunidad y año
superficie_avg = sup_filt.groupby(['comunidades_y_ciudades_autónomas', 'año']).apply(
    lambda g: pd.Series({
        'superficie_util': (g['rep_area'] * g['total']).sum() / g['total'].sum()
    })
).reset_index()
superficie_avg.rename(columns={
    'comunidades_y_ciudades_autónomas': 'region',
    'año': 'year'
}, inplace=True)


/var/folders/4q/413qpbj17ks9bgdzl52cpx9m0000gn/T/ipykernel_4797/3991255381.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  superficie_avg = sup_filt.groupby(['comunidades_y_ciudades_autónomas', 'año']).apply(


In [27]:

# Preparar datos de precios por m² (renombrar columnas para facilitar merge)
precios_df.rename(columns={'comunidades_y_ciudades_autónomas': 'region', 'año': 'year'}, inplace=True)

In [28]:
# Unir todos los datos a nivel comunidad autónoma y año
dataset1 = precios_df.merge(salario_ccaa_filtered, on=['region', 'year'], how='inner')
dataset1 = dataset1.merge(superficie_avg, on=['region', 'year'], how='inner')
dataset1 = dataset1.merge(interest_by_year, on='year', how='inner')

In [29]:
# Calcular columnas derivadas
dataset1['esfuerzo_m2'] = dataset1['precio_m2'] / dataset1['salario_mensual']
dataset1['metros_posibles'] = dataset1['salario_mensual'] / dataset1['precio_m2']
dataset1['superficie_cubierta_pct'] = dataset1['metros_posibles'] / dataset1['superficie_util']


In [30]:
# Ordenar por región y año, y guardar resultado
#dataset1.sort_values(['region', 'year'], inplace=True)
#dataset1.to_csv("dataset_maestro_ccaa.csv", index=False)

# Dataset maestro por grupos de Edad (salario mediano Decil 5)

In [27]:
# Lectura de archivo de salarios por edad
salario_edad_df = pd.read_csv("../data/clean/salario_edad_sql.csv")

# Filtrar salario por grupo de edad: decil 5 y jornada Total (excluyendo grupo "Total")
salario_edad_filtered = salario_edad_df[
    (salario_edad_df['tipo_de_jornada'] == 'Total') &
    (salario_edad_df['decil'] == '5') &
    (salario_edad_df['grupo_de_edad'] != 'Total')
].copy()
salario_edad_filtered.rename(columns={
    'grupo_de_edad': 'grupo_edad',
    'año': 'year',
    'total': 'salario_mensual'
}, inplace=True)
salario_edad_filtered = salario_edad_filtered[['grupo_edad', 'year', 'salario_mensual']]

# Calcular precio medio nacional por m² para cada año
precio_nac = precios_df.groupby('year', as_index=False)['precio_m2'].mean()
precio_nac.rename(columns={'precio_m2': 'precio_m2_nacional'}, inplace=True)

# Unir datos a nivel grupo de edad y año
dataset2 = salario_edad_filtered.merge(precio_nac, on='year', how='inner')
dataset2 = dataset2.merge(interest_by_year, on='year', how='inner')

# Calcular columnas derivadas
dataset2['metros_posibles'] = dataset2['salario_mensual'] / dataset2['precio_m2_nacional']
dataset2['esfuerzo_m2'] = dataset2['precio_m2_nacional'] / dataset2['salario_mensual']




In [ ]:
# Ordenar por año y grupo de edad, y guardar resultado
#dataset2.sort_values(['year', 'grupo_edad'], inplace=True)
#dataset2.to_csv("dataset_maestro_edad.csv", index=False)

# Datasets Maestro por decil total (promedio más real)

In [53]:
# Filtrar salario por comunidad autónoma: decil 5 y jornada Total (excluyendo Total Nacional)
salario_ccaa_total = salario_ccaa_df[
    (salario_ccaa_df['tipo_de_jornada'] == 'Total') &
    (salario_ccaa_df['decil'] == 'Total decil') &
    (salario_ccaa_df['comunidades_y_ciudades_autónomas'] != 'Total Nacional')
].copy()
salario_ccaa_total.rename(columns={
    'comunidades_y_ciudades_autónomas': 'region',
    'año': 'year',
    'total': 'salario_mensual'
}, inplace=True)
salario_ccaa_total = salario_ccaa_total[['region', 'year', 'salario_mensual']]



In [54]:
# Calcular interés medio anual (tipo fijo, naturaleza Vivienda)
interes_total = interes_df[
    (interes_df['tipo_de_interés'] == 'Fijo') &
    (interes_df['naturaleza_de_la_finca'] == 'Viviendas')
]
interest_by_year_total = interes_total.groupby('año', as_index=False)['total'].mean()
interest_by_year_total.rename(columns={'año': 'year', 'total': 'interes_medio'}, inplace=True)

# Calcular superficie útil promedio por comunidad (m²) a partir de la distribución de tamaños de vivienda
# Mapeo de categorías de superficie a valores representativos en m²
categoria_a_m2_total = {
    'Menos de 46 m2': 40,
    'Entre 46 y 75 m2': 60,
    'Entre 76 y 105 m2': 90,
    'Entre 106 y 150 m2': 128,
    'Más de 150 m2': 180
}

In [55]:
# Filtrar datos de superficie para tamaño_hogar Total, excluyendo la fila 'Total' consolidada
sup_filt_total = superficie_df[
    (superficie_df['tamaño_hogar'] == 'Total') & 
    (superficie_df['superficie_util'] != 'Total')
].copy()
sup_filt_total['rep_area'] = sup_filt_total['superficie_util'].map(categoria_a_m2)
# Calcular promedio ponderado de superficie por comunidad y año
superficie_avg_total = sup_filt_total.groupby(['comunidades_y_ciudades_autónomas', 'año']).apply(
    lambda g: pd.Series({
        'superficie_util': (g['rep_area'] * g['total']).sum() / g['total'].sum()
    })
).reset_index()
superficie_avg_total.rename(columns={
    'comunidades_y_ciudades_autónomas': 'region',
    'año': 'year'
}, inplace=True)


/var/folders/4q/413qpbj17ks9bgdzl52cpx9m0000gn/T/ipykernel_4797/1437185042.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  superficie_avg_total = sup_filt_total.groupby(['comunidades_y_ciudades_autónomas', 'año']).apply(


In [56]:

# Preparar datos de precios por m² (renombrar columnas para facilitar merge)
precios_df.rename(columns={'comunidades_y_ciudades_autónomas': 'region', 'año': 'year'}, inplace=True)

In [57]:
# Unir todos los datos a nivel comunidad autónoma y año
dataset1_total = precios_df.merge(salario_ccaa_filtered, on=['region', 'year'], how='inner')
dataset1_total = dataset1_total.merge(superficie_avg, on=['region', 'year'], how='inner')
dataset1_total = dataset1_total.merge(interest_by_year, on='year', how='inner')

In [58]:
# Calcular columnas derivadas
dataset1_total['esfuerzo_m2'] = dataset1_total['precio_m2'] / dataset1_total['salario_mensual']
dataset1_total['metros_posibles'] = dataset1_total['salario_mensual'] / dataset1_total['precio_m2']
dataset1_total['superficie_cubierta_pct'] = dataset1_total['metros_posibles'] / dataset1_total['superficie_util']


In [ ]:
# Ordenar por región y año, y guardar resultado
#dataset1_total.sort_values(['region', 'year'], inplace=True)
#dataset1_total.to_csv("dataset_total_maestro_ccaa.csv", index=False)

### dataset total decil de edad

In [60]:
# Lectura de archivo de salarios por edad
salario_edad_df_total = pd.read_csv("../data/clean/salario_edad_sql.csv")

# Filtrar salario por grupo de edad: decil 5 y jornada Total (excluyendo grupo "Total")
salario_edad_filtered_total = salario_edad_df_total[
    (salario_edad_df_total['tipo_de_jornada'] == 'Total') &
    (salario_edad_df_total['decil'] == '5') &
    (salario_edad_df_total['grupo_de_edad'] != 'Total')
].copy()
salario_edad_filtered_total.rename(columns={
    'grupo_de_edad': 'grupo_edad',
    'año': 'year',
    'total': 'salario_mensual'
}, inplace=True)
salario_edad_filtered_total = salario_edad_filtered_total[['grupo_edad', 'year', 'salario_mensual']]

# Calcular precio medio nacional por m² para cada año
precio_nac = precios_df.groupby('year', as_index=False)['precio_m2'].mean()
precio_nac.rename(columns={'precio_m2': 'precio_m2_nacional'}, inplace=True)

# Unir datos a nivel grupo de edad y año
dataset2_total = salario_edad_filtered_total.merge(precio_nac, on='year', how='inner')
dataset2_total = dataset2_total.merge(interest_by_year, on='year', how='inner')

# Calcular columnas derivadas
dataset2_total['metros_posibles'] = dataset2_total['salario_mensual'] / dataset2_total['precio_m2_nacional']
dataset2_total['esfuerzo_m2'] = dataset2_total['precio_m2_nacional'] / dataset2_total['salario_mensual']




In [ ]:
# Ordenar por año y grupo de edad, y guardar resultado
#dataset2_total.sort_values(['year', 'grupo_edad'], inplace=True)
#dataset2_total.to_csv("../data/clean/dataset_total_maestro_edad.csv", index=False)